In [ ]:
###http://cndc.produccion.gob.ar/buscadoravanzado?body_value=&field_fecha_value%5Bmin%5D%5Bdate%5D=05%2F01%2F2018&field_fecha_value%5Bmax%5D%5Bdate%5D=02%2F01%2F2021&field_resoluci_n_value=&field_dictamen_value=&field_industria_tid=All&field_tipo_de_documento_tid%5B%5D=1

scrapping of Antitrust's agency webpage

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import re
import numpy as np
import os
from urllib.parse import urljoin
###import textract
from datetime import datetime

In [3]:
url='http://cndc.produccion.gob.ar/buscadoravanzado?body_value=&field_fecha_value%5Bmin%5D%5Bdate%5D=05%2F01%2F2018&field_fecha_value%5Bmax%5D%5Bdate%5D=02%2F01%2F2021&field_resoluci_n_value=&field_dictamen_value=&field_industria_tid=All&field_tipo_de_documento_tid%5B%5D=1'
response=requests.get (url)
html= response.content
scraped=BeautifulSoup(html,'html.parser')

In [4]:
pages=np.arange (0,19,1)
nombres2=[]
nros2=[] ##dictamenes
resoluciones2=[]
fechas2=[]
tipos2=[]
industrias=[]

In [5]:
for page in pages:
    response=requests.get('http://cndc.produccion.gob.ar/buscadoravanzado?field_fecha_value%5Bmin%5D%5Bdate%5D=05/01/2018&field_fecha_value%5Bmax%5D%5Bdate%5D=02/01/2021&body_value=&field_resoluci_n_value=&field_dictamen_value=&field_industria_tid=All&field_tipo_de_documento_tid%5B0%5D=1&page='+ str(page))
    html=response.content
    scraped = BeautifulSoup(html, 'html.parser')
    ##movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    sleep(randint(2,10))
    nombres= scraped.find_all('td', class_='views-field views-field-title')
    for nombre in nombres:
        nombre=nombre.text.strip()
        nombres2.append (nombre)
    dictamenes= scraped.find_all('td', class_='views-field views-field-field-dictamen')
    for dictamen in dictamenes:
        nro= dictamen.text.strip()
        nros2.append(nro)
    resoluciones= scraped.find_all('td', class_='views-field views-field-field-resoluci-n')
    for resolucion in resoluciones:
        resolucion= resolucion.text.strip()
        resoluciones2.append(resolucion)
    fechas= scraped.find_all('td', class_='views-field views-field-field-fecha')
    for fecha in fechas:
        fecha= fecha.text.strip()
        fechas2.append(fecha)
    tipos= scraped.find_all('td', class_='views-field views-field-field-tipo-de-documento')
    for tipo in tipos:
        tipo= tipo.text.strip()
        tipos2.append(tipo)

In [6]:
CNDC = pd.DataFrame({
'Nombre dictamen': nombres2,
'Nro dictamen': nros2,
'Nro resolucion': resoluciones2,
'Fecha': fechas2,
'Tipos': tipos2 })

In [7]:
CNDC.tail()

,Nombre dictamen,Nro dictamen,Nro resolucion,Fecha,Tipos
181,“EXXONMOBIL INTERNATIONAL HOLDING INC. Y BRIDA...,15280788,324,28-05-2018,Concentraciones
182,"“CABLEVISIÓN S.A., CABLEVISIÓN HOLDING S.A., T...",30780664,374,29-06-2018,Concentraciones
183,"MOLINOS IP S.A., MOLINOS RÍO DE LA PLATA S.A.,...",28922880,363,15-06-2018,Concentraciones
184,BOLDT S.A. Y SOCIEDAD COMERCIAL DEL PLATA S.A....,4644532,113,12-06-2018,Concentraciones
185,SOCIEDAD COMERCIAL DEL PLATA S.A. Y MOLTTA CON...,294,51,10-05-2018,Concentraciones


In [8]:
CNDC.shape

(186, 5)

Downloading the Pdf's of the resolutions generated in the previous research

In [9]:
#If there is no such folder, the script will create one automatically
folder_location = r'webscraping3'
if not os.path.exists(folder_location):os.mkdir(folder_location)


In [10]:
##https://stackoverflow.com/questions/54616638/download-all-pdf-files-from-a-website-using-python 

url = 'http://cndc.produccion.gob.ar/buscadoravanzado?body_value=&field_fecha_value%5Bmin%5D%5Bdate%5D=05%2F01%2F2018&field_fecha_value%5Bmax%5D%5Bdate%5D=02%2F01%2F2021&field_resoluci_n_value=&field_dictamen_value=&field_industria_tid=All'

#La primera pagina tiene un URL distinto que no tiene numeracion. 
response = requests.get(url)
soup= BeautifulSoup(response.text, "html.parser")     
for link in soup.select("a[href$='.pdf']"):
    #Name the pdf files using the last portion of each link which are unique in this case
    filename = os.path.join(folder_location,link['href'].split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url,link['href'])).content)

In [11]:
for dictamen in nros2:
    response=requests.get('http://cndc.produccion.gob.ar/buscadoravanzado?body_value=&field_fecha_value%5Bmin%5D%5Bdate%5D=05%2F01%2F2018&field_fecha_value%5Bmax%5D%5Bdate%5D=02%2F01%2F2021&field_resoluci_n_value=&field_dictamen_value='+str(dictamen)+'&field_industria_tid=All')
    soup= BeautifulSoup(response.text, "html.parser") 
    sleep(randint(2,10))    
    for link in soup.select("a[href$='.pdf']"):
    #Name the pdf files using the last portion of each link which are unique in this case
      filename = os.path.join(folder_location,link['href'].split('/')[-1])
      with open(filename, 'wb') as f:
        f.write(requests.get(urljoin(url,link['href'])).content)
      print (dictamen)

13697837
65927058
85805552
8203915
9140455
11912489
8727633
8727876
9139980
106405137
108766154
78160782
11767380
87297003
8727355
75299479
14339278
26549336
12587310
14339478
26549263
88966103
88299165
88299560
88299051
88299240
8288202
88966214
88965867
99214667
100502933
106433714
68870532
80603182
80603182
84408759
67705334
67705334
72206293
81877084
68875066
12863824
12863824
18002632
18002632
57513963
57513963
53289753
53289753
17997165
17997165
57324758
57324758
50616902
50616902
84271629
91393066
41766327
41766327
84272684
84273383
84225198
93163334
783
84270709
80603182
80603182
78160881
68875286
57285209
66688834
78161350
67705334
67705334
53427254
53427254
12863824
12863824
18002632
18002632
57513963
57513963
53289753
53289753
17997165
17997165
57324758
57324758
53649416
56863074










50616902
50616902
51567879
41766327
41766327
5504309
68875573
68864306
53427169
40848180
9306975
6401177
33390685
57284483
39947432
40818013
50291756
39947193
40027818
66382985
52978681
42

Data mining over the Antitrust Comission's resolutions

In [12]:
#!pip install PyPDF2
import PyPDF2

In [22]:
reader = PyPDF2.PdfFileReader('/content/webscraping/CONC%201507.pdf')
reader.documentInfo

{'/CreationDate': "D:20180629135019-03'00'",
 '/Creator': 'pdfsam-console (Ver. 2.3.0e)',
 '/ModDate': "D:20180629135019-03'00'",
 '/Producer': 'iText 2.1.7 by 1T3XT'}

In [224]:
reader.numPages

217

In [222]:
num_pages=reader.numPages
count = 0
text = ""

In [23]:
while count < num_pages:
    pageObj = reader.getPage(count)
    count +=1
    text += pageObj.extractText()


In [24]:
text

'República Argentina - Poder Ejecutivo Nacional\n2018 - Año del Centenario de la Reforma Universitaria\nDictamen firma conjunta\nNúmero: \nReferencia: Conc. 1635 | Dictamen Art. 13 (a) Ley N° 25.156\n SEÑOR SECRETARIO DE COMERCIO\nElevamos para su consideración el presente dictamen, referido a la operación de concentración económica\nque tramita bajo el expediente EX-2018-24292844ŠAPN-DGD#MP del Registro del MINISTERIO DE\n\nPRODUCCIÓN, caratulado: \nﬁCONC.1635 - YARA INTERNATIONAL ASA Y VALE S.A.\n\nS/NOTIFICACIÓN ART. 8 DE LA LEY 25.156.ﬂ\nI. DESCRIPCIÓN DE LA OPERACIÓN Y ACTIVIDAD DE LAS PARTES\n\n1. El día 22 de mayo de 2018, esta Comisión Nacional recibió la notificación de una operación de\nconcentración económica, celebrada y ejecutada en el exterior, consistente en la adquisición del control\n\nexclusivo sobre VALE CUBATÃO FERTILIZANTES LTDA. (en adelante, "VALE CUBATÃO") por\n\nparte de YARA INTERNATIONAL ASA (en adelante, "YARA").\n2. La operación se implementa mediante la co

In [25]:
if text != "":
  text = text

#else:
  # text = textract.process(fileurl, method='tesseract', language='eng')

data=text

In [76]:
#pasamos el texto a minusculas, y cambiamos los saltos de lineas por espacios
data = data.lower()
data = data.replace('  ', ' ')
data = data.replace('\'', '"').replace('´', '"').replace('‘', '"').replace('’', '"')
print(data[0:100])

república argentina - poder ejecutivo nacional
2018 - año del centenario de la reforma universitaria


Necesary functions

In [13]:
def mesANumero(string):
    m = {
        'enero': "01",
        'febrero': "02",
        'marzo': "03",
        'abril': "04",
        'mayo': "05",
        'junio': "06",
        'julio': "07",
        'agosto': "08",
        'septiembre': "09",
        'setiembre': "09",
        'octubre': "10",
        'noviembre': "11",
        'diciembre': "12"
        }
    
    try:
        out = str(m[string])
        return out
    except:
        ##raise ValueError('No es un mes')
        print ('')

Closing

In [26]:
#como regex es greedy tenemos que aclarar que sea hasta la primera coincidencia con el signo de pregunta.

try:
  closing = re.search(r'el cierre de la transacción tuvo lugar el día.*? (\d+) de (\w+) de.*?(\d+)', data).groups(1)

except:
    closing = "buscar_manualmente"
    
print(closing)

('15', 'mayo', '2018')


In [29]:
if closing != "buscar_manualmente":
    añoclosing = closing[2]
    mesclosing = mesANumero(closing[1])
    diaclosing = closing[0]
    fechaclosing = datetime.strptime((añoclosing+"/"+mesclosing+"/"+diaclosing), '%Y/%m/%d').strftime('%Y/%m/%d')
else:
    fechaclosing = closing
fechaclosing

'2018/05/15'

Notified to the Agency on

In [97]:
#como regex es greedy tenemos que aclarar que sea hasta la primera coincidencia con el signo de pregunta.

try:
  notificadoel = re.search(r'el día.*? (\d+) de (\w+) de.*?(\d+), las partes notificaron', data).groups(1)

except:
    notificadoel = "buscar_manualmente"
    
print(notificadoel)

('26', 'diciembre', '2017')


In [98]:
if notificadoel != "buscar_manualmente":
    añonotificado = notificadoel[2]
    mesnotificado = mesANumero(notificadoel[1])
    dianotificado = notificadoel[0]
    fechanotificacion = datetime.strptime((añonotificado+"/"+mesnotificado+"/"+dianotificado), '%Y/%m/%d').strftime('%Y/%m/%d')
else:
    fechanotificacion = notificadoel
fechanotificacion

'2017/12/26'

In [14]:
# función para buscar las frases que no esten contenidas en comillas pero esten seguidas de las palabras dijo, dice, decía, respondío o responde
def buscar_notificacion(texto):
  match = re.search(r"(?:recibió la notificación el|notificó el|el día.*? (\d+) de (\w+) de.*?(\d+))\S?\s+((?:\S+\s*){12})", texto).groups(1)
  return match if match else "buscar_manualmente"

In [145]:
try:
  fechanotif= buscar_notificacion(data)
except:
  fechanotif = 'buscar_manualmente'

print(fechanotif)

('24', 'enero', '2018', 'es decir de modo previo al cierre.\ni.1.1 instrumentación de la operación\n')


In [130]:
if fechanotif != "buscar_manualmente":
    añonotif = fechanotif[2]
    mesnotif = mesANumero(fechanotif[1])
    dianotif = fechanotif[0]
    fecha_notif = datetime.strptime((añonotif+"/"+mesnotif+"/"+dianotif), '%Y/%m/%d').strftime('%Y/%m/%d')
else:
    fecha_notif = fechanotif
fecha_notif

'2017/12/26'

Number of proceeding

In [103]:
#como regex es greedy tenemos que aclarar que sea hasta la primera coincidencia con el signo de pregunta.

try:
  nroconc = re.search(r'conc.*? (\d+)', data).groups(1)

except:
  nroconc = 'buscar_manualmente'

print(nroconc)

('1568',)


In [ ]:
#https://towardsdatascience.com/how-to-extract-keywords-from-pdfs-and-arrange-in-order-of-their-weights-using-python-841556083341 super interesante para la ley. 
##https://www.analyticsvidhya.com/blog/2015/06/regular-expression-python/

Type of consent between the members of the Agency

In [111]:
#como regex es greedy tenemos que aclarar que sea hasta la primera coincidencia con el signo de pregunta.

try:
  firma = re.search(r'dictamen firma.*? (\w+)', data).groups(1)

except:
  firma = 'buscar_manualmente'

print(firma)

('conjunta',)


In [ ]:
#fecha de resolucion buscar a mano. 

How many times there was interaction between the client and the Agency

In [15]:
# función para buscar las frases que no esten contenidas en comillas pero esten seguidas de las palabras dijo, dice, decía, respondío o responde
def rfis(texto):
  match = re.findall (r"(?: el día.*? (\d+) de (\w+) de.*?(\d+))\S?\s+((?:\S+\s*){12})", texto)
  return match if match else "buscar_manualmente"

In [137]:
try:
  interaction= rfis(data)
except:
  interaction = 'buscar_manualmente'

print(interaction)

[('26', 'diciembre', '2017', 'esta comisión nacional recibió la notificación de una operación de concentración\neconómica, '), ('31', 'octubre', '2016', '27. el día 7 de noviembre de 2016, las partes solicitaron a '), ('22', 'diciembre', '2017', '310. el día 26 de diciembre de 2017, las partes notificaron la '), ('19', 'diciembre', '2017', 'fue dictada la resolución n° 940/17, que dispuso sujetar la operación descripta '), ('22', 'diciembre', '2017', '23. el día 26 de diciembre de 2017, las partes notificaron la '), ('26', 'enero', '2018', 'šy tras analizar las presentaciones efectuadasš, esta comisión nacional consideró que\nla '), ('26', 'enero', '2018', 'se ordenó la digitalización del expte. nº s01: 0508638/2016 del\nregistro del '), ('26', 'diciembre', '2017', 'consiste en la\nadquisición del control exclusivo sobre las firmas avian lineas ')]


In [201]:
len (interaction)

8

OS ordering the files names

Automatization

In [16]:
#Empty lists

cl=[]
notif=[]
conc=[]
firm=[]
inter=[]


#Loop

for foldername,subfolders,files in os.walk(r"/content/webscraping3"):
    files= sorted (files)
    for file in files:
        print (file)
        # open the pdf file
        reader = PyPDF2.PdfFileReader(os.path.join(foldername,file))
        
        num_pages=reader.numPages
        count = 0
        text = ""

        while count < num_pages:
          pageObj = reader.getPage(count)
          count +=1
          try:
            text += pageObj.extractText()
          except:
            print (count)
                      
       
        if text != "":
          text = text

      ##  else:
       ##   text = textract.process(fileurl, method='tesseract', language='sp')
        
        data=text
        
        #pasamos el texto a minusculas, y cambiamos los saltos de lineas por espacios
        data = data.lower()
        data = data.replace('  ', ' ')
        data = data.replace('\'', '"').replace('´', '"').replace('‘', '"').replace('’', '"')
        data= data.replace('\n','')
       
        #Closing
        
        try:
          closing = re.search(r'el cierre de la transacción tuvo lugar el día.*? (\d+) de (\w+) de.*?(\d+)', data).groups(1)

        except:
            closing = "buscar_manualmente"

        if closing != "buscar_manualmente":
          añoclosing = closing[2]
          mesclosing = mesANumero(closing[1])
          diaclosing = closing[0]
          fechaclosing = datetime.strptime((añoclosing+"/"+mesclosing+"/"+diaclosing), '%Y/%m/%d').strftime('%Y/%m/%d')
        else:
          fechaclosing = closing
         
        try: 
          cl.append(fechaclosing)
        except:
          cl.append(' ') 

        ##Notificacion

        try:
          fechanotif= buscar_notificacion(data)
        except:
          fechanotif = 'buscar_manualmente'
        
        if fechanotif != "buscar_manualmente":
          try:
            añonotif = fechanotif[2]
            mesnotif = mesANumero(fechanotif[1])
            dianotif = fechanotif[0]
            fecha_notif = datetime.strptime((añonotif+"/"+mesnotif+"/"+dianotif), '%Y/%m/%d').strftime('%Y/%m/%d')
          except: 
            fecha_notif = fechanotif
        else:
          fecha_notif = fechanotif
          
        try: 
          notif.append(fecha_notif)
        except:
          notif.append(' ')     

      ###Numero de concentracion
          
        try: 
          nroconc = re.search(r'conc.*? (\d+)', data).groups(1)

        except:
          nroconc = 'buscar_manualmente'

        try: 
          conc.append(nroconc)
        except:
          conc.append(' ')

      ##Consent

        try:
          firma = re.search(r'dictamen firma.*? (\w+)', data).groups(1)

        except:
          firma = 'buscar_manualmente'

        try: 
          firm.append(firma)
        except:
          firm.append(' ')
        

    #interaction

        try:
          interaction= rfis(data)
        except:
          interaction = 'buscar_manualmente'

        try: 
          inter.append (len (interaction))
        except:
          inter.append(' ')       

1568%20completa.pdf
1576-Completo.pdf
1635%20COMPLETO.pdf
C.%201614%20-%202021.pdf
C.%201647%20-%202021.pdf
C.%201697%20-%202021.pdf
CONC%201119.pdf
CONC%201130.pdf
CONC%201144.pdf
CONC%201149.pdf
CONC%201187.pdf
CONC%201239.pdf
CONC%201246.pdf
CONC%201269.pdf
CONC%201270.pdf
CONC%201275.pdf
CONC%201279.pdf
CONC%201304.pdf
CONC%201317.pdf
CONC%201324.pdf
CONC%201325.pdf
CONC%201345.pdf
CONC%201346.pdf
CONC%201352%202019.pdf
CONC%201362.pdf
CONC%201365.pdf
CONC%201366.pdf
CONC%201392.pdf
CONC%201405.pdf
CONC%201416.pdf
CONC%201438.pdf
CONC%201445.pdf
CONC%201447.pdf
CONC%201454.pdf
CONC%201469%202019.pdf
CONC%201480.pdf
CONC%201507.pdf
13
CONC%201511.pdf
CONC%201512.pdf
CONC%201514%20%282%29.pdf
CONC%201518.pdf
CONC%201519%202019.pdf
CONC%201521%202019.pdf
CONC%201528.pdf
CONC%201559.pdf
CONC%201562%202019.pdf
CONC%201563%202020.pdf
CONC%201565.pdf
CONC%201573.pdf
CONC%201574%202019.pdf
CONC%201578.pdf
CONC%201579.pdf
CONC%201580.pdf
CONC%201585.pdf
CONC%201586.pdf
CONC%201592%202019.pd

In [17]:
leyvieja = pd.DataFrame({
'Nro concentracion': conc,
'Closing': cl,
'Consenso': firm,
'Interaccion con el notificante': inter})


In [18]:
leyvieja

,Nro concentracion,Closing,Consenso,Interaccion con el notificante
0,"(1568,)",2016/10/31,"(conjuntanúmero,)",10
1,"(25,)",buscar_manualmente,"(conjuntanúmero,)",18
2,"(1635,)",2018/05/15,"(conjuntanúmero,)",10
3,"(4,)",buscar_manualmente,"(conjunta,)",12
4,"(10,)",buscar_manualmente,"(conjunta,)",7
...,...,...,...,...
188,"(9,)",2019/07/24,"(conjunta,)",5
189,"(8,)",buscar_manualmente,buscar_manualmente,21
190,"(887,)",buscar_manualmente,buscar_manualmente,2
191,"(9,)",buscar_manualmente,"(conjunta,)",7


In [20]:
from google.colab import files
leyvieja.to_csv('leyvieja.csv') 
files.download('leyvieja.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>